In [27]:
#!/usr/bin/env python3

import os
import sys
import glob
import csv
import time
import math
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple, Optional, Dict, Any
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import subprocess
import importlib

for pkg in ("networkx", "gurobipy", "pandas", "matplotlib", "seaborn"):
    try:
        importlib.import_module(pkg)
    except ImportError:
        print(f"Installing {pkg}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

import networkx as nx
import gurobipy as gp 
from gurobipy import GRB

plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
sns.set_palette("husl")

print("=" * 60)
print("Treedepth ILP Integrated Experiment System")
print("=" * 60)
print(f"Experiment start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Gurobi version: {gp.gurobi.version()}")
print(f"Available CPU cores: {os.cpu_count()}")
print("=" * 60)


Treedepth ILP Integrated Experiment System
Experiment start time: 2025-08-21 15:42:53
Gurobi version: (12, 0, 3)
Available CPU cores: 32


In [28]:
EXPERIMENTAL_CONFIG = {
    "TIMEOUT": 120,
    "MAX_NODES": 50,
    "MAX_EDGES": 2000,
    "THREADS": max(1, os.cpu_count() // 4),
    "RANDOM_SEED": 42,
    "MIPSTART_RUNS": 10,
}

SKIP_LIST = {"Paley17", "Holt", "Watsin","McGee","Kittell"}

DATASET_PATHS = {
    "famous": "inputs/famous/*.edge",
    "standard": "inputs/standard/*.edge"
}

OUTPUT_FILES = {
    "baseline": "results_4_2_baseline_ilp.csv",
    "preprocessing": "results_4_3_preprocessing.csv", 
    "components": "results_4_4_components.csv",
    "variants": "results_4_5_variants.csv",
    "final_comparison": "results_4_6_ilp_vs_sat.csv"
}

STANDARD_CSV_HEADERS = [
    "experiment",
    "instance",
    "dataset",
    "n",
    "m",
    "treedepth",
    "time",
    "status",
    "timeout",
    "memory_usage_mb",
    "preprocessing_enabled",
    "mipstart_enabled",
    "lazy_constraints_enabled",
    "gurobi_presolve",
    "encoding_variant",
    "gurobi_objval",
    "gurobi_mip_gap",
    "gurobi_node_count",
    "problem_variables",
    "problem_constraints",
    "graph_density",
    "graph_components",
    "preprocessing_reduction",
]

print("Experiment configuration:")
for key, value in EXPERIMENTAL_CONFIG.items():
    print(f"  {key}: {value}")

print(f"\nOutput files:")
for exp, filename in OUTPUT_FILES.items():
    print(f"  {exp}: {filename}")

print(f"\nSkipped instances: {len(SKIP_LIST)}")
print(f"CSV standard columns: {len(STANDARD_CSV_HEADERS)}")

os.makedirs("results", exist_ok=True)
print("\n✓ Experiment configuration complete")


Experiment configuration:
  TIMEOUT: 120
  MAX_NODES: 50
  MAX_EDGES: 2000
  THREADS: 8
  RANDOM_SEED: 42
  MIPSTART_RUNS: 10

Output files:
  baseline: results_4_2_baseline_ilp.csv
  preprocessing: results_4_3_preprocessing.csv
  components: results_4_4_components.csv
  variants: results_4_5_variants.csv
  final_comparison: results_4_6_ilp_vs_sat.csv

Skipped instances: 5
CSV standard columns: 23

✓ Experiment configuration complete


In [29]:
GOLDEN_STANDARD = {
    "Diamond": 3, "Bull": 3, "Butterfly": 3, "Prism": 5, "Moser": 5,
    "Wagner": 6, "Pmin": 5, "3x3-grid": 5, "Petersen": 6, "Herschel": 5,
    "Grotzsch": 7, "Goldner": 5, "Durer": 7, "Franklin": 7, "Frucht": 6,
    "Tietze": 7, "Chvatal": 8, "Paley13": 10, "Poussin": 9, "4x4-grid": 7,
    "Sousselier": 8, "Hoffman": 8, "Clebsch": 10, "Shrikhande": 11, "Errera": 10,
    "Pappus": 8, "Robertson": 10, "Desargues": 9, "Dodecahedron": 9, 
    "FlowerSnark": 9, "Folkman": 9, "Brinkmann": 11, "Kittell": 12, 
    "McGee": 11, "Nauru": 10
}

def validate_result(instance_name: str, computed_value: Optional[int], is_timeout: bool = False) -> Dict[str, Any]:
    validation_result = {
        "instance": instance_name,
        "golden_value": GOLDEN_STANDARD.get(instance_name),
        "computed_value": computed_value,
        "is_timeout": is_timeout,
        "is_correct": False,
        "error_type": None,
        "validation_status": "unknown"
    }
    
    if instance_name not in GOLDEN_STANDARD:
        validation_result["validation_status"] = "no_golden_standard"
        validation_result["is_correct"] = None
        return validation_result
    
    golden_value = GOLDEN_STANDARD[instance_name]
    
    if computed_value is None:
        validation_result["error_type"] = "failed_to_solve"
        validation_result["validation_status"] = "failed"
        return validation_result
    
    if is_timeout:
        validation_result["is_correct"] = False
        validation_result["error_type"] = "timeout_not_optimal"
        validation_result["validation_status"] = "error"
        return validation_result
    
    if computed_value == golden_value:
        validation_result["is_correct"] = True
        validation_result["validation_status"] = "correct_optimal"
    else:
        validation_result["is_correct"] = False
        if computed_value < golden_value:
            validation_result["error_type"] = "underestimate"
        else:
            validation_result["error_type"] = "overestimate"
        validation_result["validation_status"] = "error"
    
    return validation_result

def analyze_validation_results(validation_results: List[Dict[str, Any]]) -> Dict[str, Any]:
    total = len(validation_results)
    if total == 0:
        return {}
    
    testable_results = [r for r in validation_results if r["validation_status"] != "no_golden_standard"]
    testable_count = len(testable_results)
    
    if testable_count == 0:
        return {"total_instances": total, "testable_instances": 0, "accuracy_rate": 0}
    
    correct_count = len([r for r in testable_results if r["is_correct"]])
    error_count = len([r for r in testable_results if r["validation_status"] == "error"])
    failed_count = len([r for r in testable_results if r["validation_status"] == "failed"])
    timeout_errors = len([r for r in testable_results if r["error_type"] == "timeout_not_optimal"])
    
    error_types = {}
    for result in testable_results:
        if result["error_type"]:
            error_types[result["error_type"]] = error_types.get(result["error_type"], 0) + 1
    
    analysis = {
        "total_instances": total,
        "testable_instances": testable_count,
        "correct_count": correct_count,
        "error_count": error_count,
        "failed_count": failed_count,
        "timeout_errors": timeout_errors,
        "accuracy_rate": correct_count / testable_count,
        "error_rate": error_count / testable_count,
        "failure_rate": failed_count / testable_count,
        "timeout_error_rate": timeout_errors / testable_count,
        "error_types": error_types
    }
    
    return analysis

def calculate_method_metrics(results: List[Dict], method_name: str = ""):
    if not results:
        return {
            "method_name": method_name,
            "accuracy_rate": 0,
            "success_rate": 0,
            "optimal_rate": 0,
            "avg_time": float('inf'),
            "testable_instances": 0,
            "correct_instances": 0,
            "timeout_instances": 0,
            "is_valid": False
        }
    
    validation_results = []
    for r in results:
        validation = validate_result(r.get("instance", ""), r.get("treedepth"), r.get("timeout", False))
        validation_results.append(validation)
    
    analysis = analyze_validation_results(validation_results)
    accuracy_rate = analysis.get("accuracy_rate", 0)
    testable_instances = analysis.get("testable_instances", 0)
    correct_instances = analysis.get("correct_count", 0)
    
    success_results = [r for r in results if r.get("status") in ["optimal", "timeout"]]
    success_rate = len(success_results) / len(results) if results else 0
    
    optimal_results = [r for r in results if r.get("status") == "optimal"]
    optimal_rate = len(optimal_results) / len(results) if results else 0
    
    timeout_results = [r for r in results if r.get("status") == "timeout"]
    timeout_instances = len(timeout_results)
    
    successful_times = [r.get("time", float('inf')) for r in success_results if "time" in r and r["time"] is not None]
    avg_time = np.mean(successful_times) if successful_times else float('inf')
    
    return {
        "method_name": method_name,
        "accuracy_rate": accuracy_rate,
        "success_rate": success_rate,
        "optimal_rate": optimal_rate,
        "avg_time": avg_time,
        "testable_instances": testable_instances,
        "correct_instances": correct_instances,
        "timeout_instances": timeout_instances,
        "error_instances": analysis.get("error_count", 0),
        "failed_instances": analysis.get("failed_count", 0),
        "error_types": analysis.get("error_types", {}),
        "is_valid": testable_instances > 0
    }

def select_best_method_tiered(methods_metrics: List[Dict], accuracy_tolerance: float = 0.02):
    if not methods_metrics:
        return None
    
    valid_methods = [m for m in methods_metrics if m["is_valid"]]
    
    if not valid_methods:
        print("⚠️ Warning: No method has testable instances")
        return None
    
    valid_methods.sort(key=lambda x: x["accuracy_rate"], reverse=True)
    best_accuracy = valid_methods[0]["accuracy_rate"]
    
    top_accuracy_methods = [m for m in valid_methods 
                           if abs(m["accuracy_rate"] - best_accuracy) <= accuracy_tolerance]
    
    print(f"\n🎯 First round (Accuracy - only optimal within timeout):")
    print(f"   Best accuracy: {best_accuracy:.1%}")
    print(f"   Methods within tolerance: {len(top_accuracy_methods)}")
    for m in top_accuracy_methods:
        print(f"     {m['method_name']}: Accuracy {m['accuracy_rate']:.1%}, Optimal rate {m['optimal_rate']:.1%}")
    
    if len(top_accuracy_methods) == 1:
        best_method = top_accuracy_methods[0]
        print(f"\n Selection: {best_method['method_name']} (unique highest accuracy)")
        return best_method
    
    print(f"\n Second round (Optimal rate):")
    top_accuracy_methods.sort(key=lambda x: x["optimal_rate"], reverse=True)
    
    best_optimal_rate = top_accuracy_methods[0]["optimal_rate"]
    best_optimal_candidates = [m for m in top_accuracy_methods 
                              if abs(m["optimal_rate"] - best_optimal_rate) <= 0.02]
    
    for m in best_optimal_candidates[:3]:
        time_str = f"{m['avg_time']:.2f}s" if m['avg_time'] != float('inf') else "∞"
        print(f"     {m['method_name']}: Optimal rate {m['optimal_rate']:.1%}, Avg time {time_str}")
    
    best_method = min(best_optimal_candidates, key=lambda x: x["avg_time"])
    
    print(f"\n🏆 Final selection: {best_method['method_name']}")
    print(f"   Accuracy: {best_method['accuracy_rate']:.1%} (only optimal within timeout)")
    print(f"   Optimal rate: {best_method['optimal_rate']:.1%}")
    time_str = f"{best_method['avg_time']:.2f}s" if best_method['avg_time'] != float('inf') else "∞"
    print(f"   Avg time: {time_str}")
    
    return best_method

def display_validation_summary(validation_results: List[Dict[str, Any]], method_name: str = ""):
    analysis = analyze_validation_results(validation_results)
    
    print(f"\n{'='*50}")
    print(f"{method_name} Validation Results (timeout = error)")
    print(f"{'='*50}")
    print(f"Total instances: {analysis['total_instances']}")
    print(f"Testable instances: {analysis['testable_instances']} (with golden standard)")
    print(f" Correct: {analysis['correct_count']} (only optimal within timeout)")
    print(f" Errors: {analysis['error_count']}")
    print(f"    - Timeout errors: {analysis['timeout_errors']}")
    print(f" Failed: {analysis['failed_count']}")
    print(f" Accuracy: {analysis['accuracy_rate']:.1%} (only optimal within timeout)")
    
    if analysis['error_types']:
        print(f"\nError types:")
        for error_type, count in analysis['error_types'].items():
            error_desc = {
                "timeout_not_optimal": "Timeout (upper bound not valid)",
                "underestimate": "Underestimate", 
                "overestimate": "Overestimate",
                "failed_to_solve": "Failed to solve"
            }
            desc = error_desc.get(error_type, error_type)
            print(f"  {desc}: {count}")
    
    error_instances = [r for r in validation_results if r["validation_status"] == "error"]
    if error_instances:
        print(f"\n Error details:")
        for err in error_instances[:5]:
            if err["error_type"] == "timeout_not_optimal":
                print(f"  {err['instance']}: Timeout upper bound {err['computed_value']} (True: {err['golden_value']})")
            else:
                print(f"  {err['instance']}: Computed={err['computed_value']}, True={err['golden_value']}, Error={err['error_type']}")
        if len(error_instances) > 5:
            print(f"  ... {len(error_instances)-5} more errors")
    
    return analysis

print(" Golden standard validation system loaded (timeout = error)")
print(f" Contains {len(GOLDEN_STANDARD)} instances with correct treedepth")
print(" Validation rule: Only optimal solutions within time are correct")
print(" Timeout upper bounds are errors")

print(f"\n Golden standard data:")
for i, (name, value) in enumerate(list(GOLDEN_STANDARD.items())[:10]):
    print(f"  {name}: {value}")
if len(GOLDEN_STANDARD) > 10:
    print(f"  ... total {len(GOLDEN_STANDARD)} instances")

print("\n Strict validation system ready (timeout = error)")


 Golden standard validation system loaded (timeout = error)
 Contains 35 instances with correct treedepth
 Validation rule: Only optimal solutions within time are correct
 Timeout upper bounds are errors

 Golden standard data:
  Diamond: 3
  Bull: 3
  Butterfly: 3
  Prism: 5
  Moser: 5
  Wagner: 6
  Pmin: 5
  3x3-grid: 5
  Petersen: 6
  Herschel: 5
  ... total 35 instances

 Strict validation system ready (timeout = error)


In [30]:
def read_edge_file(filename: str) -> Tuple[List[Tuple[int, int]], int]:
    edges = []
    n_declared = 0
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith('c'):
                    continue
                if line.startswith('p '):
                    parts = line.split()
                    if len(parts) >= 3:
                        n_declared = int(parts[2])
                elif line.startswith('e '):
                    parts = line.split()
                    if len(parts) >= 3:
                        u, v = int(parts[1]) - 1, int(parts[2]) - 1
                        edges.append((u, v))
        return edges, n_declared
    except Exception as e:
        raise Exception(f"Failed to read file: {e}")

def create_graph_from_edges(edges: List[Tuple[int, int]], n_nodes: int) -> nx.Graph:
    G = nx.Graph()
    G.add_nodes_from(range(n_nodes))
    G.add_edges_from(edges)
    return G

def get_graph_statistics(G: nx.Graph) -> Dict[str, Any]:
    n = G.number_of_nodes()
    m = G.number_of_edges()
    return {
        "n": n,
        "m": m,
        "density": m / (n * (n - 1) / 2) if n > 1 else 0,
        "components": nx.number_connected_components(G),
        "is_connected": nx.is_connected(G),
        "avg_degree": 2 * m / n if n > 0 else 0,
        "max_degree": max(dict(G.degree()).values()) if n > 0 else 0
    }

def load_test_instances() -> List[Tuple[str, str, nx.Graph, Dict[str, Any]]]:
    instances = []
    files = []
    for dataset, pattern in DATASET_PATHS.items():
        files.extend([(f, dataset) for f in glob.glob(pattern)])
    files.sort(key=lambda x: x[0])
    print(f"Found {len(files)} instance files")
    for filepath, dataset in files:
        instance = os.path.splitext(os.path.basename(filepath))[0]
        if instance in SKIP_LIST:
            continue
        try:
            edges, n_declared = read_edge_file(filepath)
            G = create_graph_from_edges(edges, n_declared)
            stats = get_graph_statistics(G)
            if stats["n"] > EXPERIMENTAL_CONFIG["MAX_NODES"] or stats["m"] > EXPERIMENTAL_CONFIG["MAX_EDGES"]:
                continue
            instances.append((instance, dataset, G, stats))
        except Exception as e:
            print(f"  Warning: skip instance {instance}: {e}")
            continue
    print(f"Loaded: {len(instances)} valid instances")
    small = len([i for i in instances if i[3]["n"] <= 10])
    medium = len([i for i in instances if 10 < i[3]["n"] <= 20])
    large = len([i for i in instances if i[3]["n"] > 20])
    print(f"  Small (<=10 nodes): {small}")
    print(f"  Medium (11-20 nodes): {medium}")
    print(f"  Large (>20 nodes): {large}")
    return instances

def estimate_bounds(G: nx.Graph) -> Tuple[int, int]:
    n = G.number_of_nodes()
    if n <= 1:
        return n, n
    LB = 1
    for comp in nx.connected_components(G):
        H = G.subgraph(comp)
        if H.number_of_nodes() == 1:
            LBc = 1
        else:
            src = next(iter(comp))
            dist1 = nx.single_source_shortest_path_length(G, src)
            far = max(dist1, key=dist1.get)
            dist2 = nx.single_source_shortest_path_length(G, far)
            diam = max(dist2.values())
            LBc = max(1, math.ceil(math.log2(diam + 1)))
        LB = max(LB, LBc)
    def dfs_height(component_nodes: List[int]) -> int:
        visited = set()
        maxh = 1
        for start in component_nodes:
            if start in visited:
                continue
            stack = [(start, 1, None)]
            while stack:
                v, d, parent = stack.pop()
                if v in visited:
                    continue
                visited.add(v)
                maxh = max(maxh, d)
                for w in G.neighbors(v):
                    if w != parent:
                        stack.append((w, d + 1, v))
        return maxh
    UB = 0
    for comp in nx.connected_components(G):
        UB = max(UB, dfs_height(list(comp)))
    UB = min(UB, n)
    return LB, UB

print("Testing utility functions")
test_instances = load_test_instances()
print(f"Loaded {len(test_instances)} test instances")

print("\nFirst 5 instances stats:")
for i, (name, dataset, G, stats) in enumerate(test_instances[:5]):
    lb, ub = estimate_bounds(G)
    print(f"  {name}: n={stats['n']}, m={stats['m']}, density={stats['density']:.3f}, estimated_bounds=[{lb}, {ub}]")


Testing utility functions
Found 40 instance files
Loaded: 33 valid instances
  Small (<=10 nodes): 9
  Medium (11-20 nodes): 22
  Large (>20 nodes): 2
Loaded 33 test instances

First 5 instances stats:
  Brinkmann: n=21, m=42, density=0.200, estimated_bounds=[2, 21]
  Bull: n=5, m=5, density=0.500, estimated_bounds=[2, 4]
  Butterfly: n=5, m=6, density=0.600, estimated_bounds=[2, 3]
  Chvatal: n=12, m=24, density=0.364, estimated_bounds=[2, 12]
  Clebsch: n=16, m=40, density=0.333, estimated_bounds=[2, 16]


In [31]:
def apex_vertices(G: nx.Graph) -> Tuple[nx.Graph, int]:
    if G.number_of_nodes() <= 1:
        return G.copy(), 0
    n = G.number_of_nodes()
    to_remove = [v for v, degree in G.degree() if degree == n - 1]
    G_processed = G.copy()
    G_processed.remove_nodes_from(to_remove)
    G_processed = nx.convert_node_labels_to_integers(G_processed, first_label=0, ordering="default")
    return G_processed, len(to_remove)

def degree_one_reduction(G: nx.Graph) -> nx.Graph:
    if G.number_of_nodes() <= 1:
        return G.copy()
    G_processed = G.copy()
    to_remove = set()
    for u in G_processed.nodes():
        degree_one_neighbors = [v for v in G_processed.neighbors(u) if G_processed.degree(v) == 1]
        if len(degree_one_neighbors) > 1:
            to_remove.update(degree_one_neighbors[1:])
    G_processed.remove_nodes_from(to_remove)
    G_processed = nx.convert_node_labels_to_integers(G_processed, first_label=0, ordering="default")
    return G_processed

def preprocess_graph(G: nx.Graph, enable_preprocessing: bool = True) -> Tuple[nx.Graph, int, Dict[str, Any]]:
    original_stats = get_graph_statistics(G)
    preprocess_stats = {
        "original_nodes": original_stats["n"],
        "original_edges": original_stats["m"],
        "degree_one_removed": 0,
        "apex_removed": 0,
        "final_nodes": 0,
        "final_edges": 0,
        "reduction_rate": 0.0,
        "preprocessing_enabled": enable_preprocessing
    }
    if not enable_preprocessing:
        preprocess_stats.update({
            "final_nodes": original_stats["n"],
            "final_edges": original_stats["m"]
        })
        return G.copy(), 0, preprocess_stats
    G_step1 = degree_one_reduction(G)
    degree_one_removed = original_stats["n"] - G_step1.number_of_nodes()
    G_step2, apex_removed = apex_vertices(G_step1)
    final_stats = get_graph_statistics(G_step2)
    reduction_rate = 1.0 - (final_stats["n"] / original_stats["n"]) if original_stats["n"] > 0 else 0.0
    preprocess_stats.update({
        "degree_one_removed": degree_one_removed,
        "apex_removed": apex_removed,
        "final_nodes": final_stats["n"],
        "final_edges": final_stats["m"],
        "reduction_rate": reduction_rate
    })
    return G_step2, apex_removed, preprocess_stats

def analyze_preprocessing_effectiveness():
    print("Analyzing preprocessing effectiveness...")
    preprocessing_analysis = []
    for instance, dataset, G, original_stats in test_instances:
        G_no_prep, apex_buf_no, stats_no = preprocess_graph(G, enable_preprocessing=False)
        G_prep, apex_buf_yes, stats_yes = preprocess_graph(G, enable_preprocessing=True)
        analysis_record = {
            "instance": instance,
            "dataset": dataset,
            "original_n": original_stats["n"],
            "original_m": original_stats["m"],
            "no_prep_n": stats_no["final_nodes"],
            "no_prep_m": stats_no["final_edges"],
            "prep_n": stats_yes["final_nodes"],
            "prep_m": stats_yes["final_edges"],
            "degree_one_removed": stats_yes["degree_one_removed"],
            "apex_removed": stats_yes["apex_removed"],
            "reduction_rate": stats_yes["reduction_rate"],
            "effective": stats_yes["reduction_rate"] > 0.0
        }
        preprocessing_analysis.append(analysis_record)
    effective_count = sum(1 for r in preprocessing_analysis if r["effective"])
    total_instances = len(preprocessing_analysis)
    avg_reduction = np.mean([r["reduction_rate"] for r in preprocessing_analysis])
    print("Preprocessing analysis result:")
    print(f"  Total instances: {total_instances}")
    print(f"  Effective preprocessing: {effective_count} ({effective_count/total_instances*100:.1f}%)")
    print(f"  Average node reduction rate: {avg_reduction:.3f}")
    effective_cases = [r for r in preprocessing_analysis if r["reduction_rate"] > 0.1]
    if effective_cases:
        print("\nInstances with more than 10% node reduction:")
        for case in sorted(effective_cases, key=lambda x: x["reduction_rate"], reverse=True)[:5]:
            print(f"  {case['instance']}: {case['original_n']}→{case['prep_n']} (reduction {case['reduction_rate']:.1%})")
    return preprocessing_analysis

print("Testing preprocessing functions...")

demo_instance = None
for instance, dataset, G, stats in test_instances:
    if 8 <= stats["n"] <= 15:
        demo_instance = (instance, dataset, G, stats)
        break

if demo_instance:
    name, dataset, G, stats = demo_instance
    print(f"\nDemo instance: {name}")
    print(f"  Original: n={stats['n']}, m={stats['m']}")
    G_no, apex_no, stats_no = preprocess_graph(G, enable_preprocessing=False)
    print(f"  Without preprocessing: n={stats_no['final_nodes']}, m={stats_no['final_edges']}")
    G_yes, apex_yes, stats_yes = preprocess_graph(G, enable_preprocessing=True)
    print(f"  With preprocessing: n={stats_yes['final_nodes']}, m={stats_yes['final_edges']}")
    print(f"  Degree-one removed: {stats_yes['degree_one_removed']}")
    print(f"  Apex removed: {stats_yes['apex_removed']}")
    print(f"  Reduction rate: {stats_yes['reduction_rate']:.1%}")

preprocessing_effectiveness = analyze_preprocessing_effectiveness()

print("\n Preprocessing functions test complete")


Testing preprocessing functions...

Demo instance: Chvatal
  Original: n=12, m=24
  Without preprocessing: n=12, m=24
  With preprocessing: n=12, m=24
  Degree-one removed: 0
  Apex removed: 0
  Reduction rate: 0.0%
Analyzing preprocessing effectiveness...
Preprocessing analysis result:
  Total instances: 33
  Effective preprocessing: 2 (6.1%)
  Average node reduction rate: 0.021

Instances with more than 10% node reduction:
  Diamond: 4→2 (reduction 50.0%)
  Butterfly: 5→4 (reduction 20.0%)

 Preprocessing functions test complete


In [32]:
import random

def build_deterministic_dfs_mipstart(G: nx.Graph) -> Dict[str, Any]:
    nodes = list(G.nodes())
    if not nodes:
        return {"H": 0, "depth": {}, "parent": {}, "ancestor": {}, "root": {}}
    parent = {v: None for v in nodes}
    depth = {v: None for v in nodes}
    def dfs(root: int):
        stack = [(root, 1, None)]
        while stack:
            v, d, p = stack.pop()
            if depth[v] is not None:
                continue
            depth[v] = d
            parent[v] = p
            neighbors = sorted(G.neighbors(v))
            for w in reversed(neighbors):
                if depth[w] is None:
                    stack.append((w, d + 1, v))
    for comp in nx.connected_components(G):
        root = min(comp)
        dfs(root)
    H = max(depth.values()) if depth else 0
    def is_ancestor(u, v):
        cur = parent[v]
        while cur is not None:
            if cur == u:
                return True
            cur = parent[cur]
        return False
    ancestor = {(u, v): int(u != v and is_ancestor(u, v)) for u in nodes for v in nodes}
    root = {v: int(parent[v] is None) for v in nodes}
    return {"H": H, "depth": depth, "parent": parent, "ancestor": ancestor, "root": root}

def build_randomized_dfs_mipstart(G: nx.Graph, seed: Optional[int] = None) -> Dict[str, Any]:
    if seed is not None:
        random.seed(seed)
    nodes = list(G.nodes())
    if not nodes:
        return {"H": 0, "depth": {}, "parent": {}, "ancestor": {}, "root": {}}
    parent = {v: None for v in nodes}
    depth = {v: None for v in nodes}
    def randomized_dfs(root: int):
        stack = [(root, 1, None)]
        while stack:
            v, d, p = stack.pop()
            if depth[v] is not None:
                continue
            depth[v] = d
            parent[v] = p
            neighbors = list(G.neighbors(v))
            random.shuffle(neighbors)
            for w in neighbors:
                if depth[w] is None:
                    stack.append((w, d + 1, v))
    for comp in nx.connected_components(G):
        comp_list = list(comp)
        root = random.choice(comp_list)
        randomized_dfs(root)
    H = max(depth.values()) if depth else 0
    def is_ancestor(u, v):
        cur = parent[v]
        while cur is not None:
            if cur == u:
                return True
            cur = parent[cur]
        return False
    ancestor = {(u, v): int(u != v and is_ancestor(u, v)) for u in nodes for v in nodes}
    root = {v: int(parent[v] is None) for v in nodes}
    return {"H": H, "depth": depth, "parent": parent, "ancestor": ancestor, "root": root}

def build_multiple_mipstarts(G: nx.Graph, num_runs: int = 10) -> Dict[str, Any]:
    best_solution = None
    best_height = float('inf')
    for run in range(num_runs):
        try:
            solution = build_randomized_dfs_mipstart(G, seed=run)
            height = solution["H"]
            if height < best_height:
                best_height = height
                best_solution = solution
        except Exception as e:
            continue
    if best_solution is None:
        best_solution = build_deterministic_dfs_mipstart(G)
    return best_solution

def setup_gurobi_model(model: gp.Model, component_config: Dict[str, Any]):
    model.Params.OutputFlag = 0
    model.Params.TimeLimit = EXPERIMENTAL_CONFIG["TIMEOUT"]
    model.Params.Threads = EXPERIMENTAL_CONFIG["THREADS"]
    model.Params.Cuts = 2
    model.Params.Symmetry = 2
    model.Params.NodefileStart = 0.5
    if component_config.get("mipstart_enabled", True):
        model.Params.MIPFocus = 1
        model.Params.Heuristics = 0.4
    else:
        model.Params.StartNumber = 0
        model.Params.Heuristics = 0.1
    presolve_level = component_config.get("gurobi_presolve", 2)
    model.Params.Presolve = presolve_level
    if component_config.get("lazy_constraints_enabled", True):
        model.Params.LazyConstraints = 1
    else:
        model.Params.LazyConstraints = 0
    model.Params.MIPFocus  = 1
    model.Params.Cuts      = 2
    model.Params.Symmetry  = 2

def lazy_transitivity_callback(model: gp.Model, where):
    if where != GRB.Callback.MIPSOL:
        return
    a = model._a
    nodes = model._nodes
    aval = {}
    try:
        for i in nodes:
            for j in nodes:
                if i != j:
                    aval[(i, j)] = model.cbGetSolution(a[i, j])
    except:
        return
    violations_added = 0
    for i in nodes:
        for j in nodes:
            if i == j or aval.get((i, j), 0) < 0.5:
                continue
            for k in nodes:
                if k != i and k != j:
                    if (aval.get((j, k), 0) > 0.5 and aval.get((i, k), 0) < 0.5):
                        model.cbLazy(a[i, k] >= a[i, j] + a[j, k] - 1)
                        violations_added += 1
                        if violations_added >= 10:
                            return

print("Testing MIPStart generation...")

if test_instances:
    test_name, test_dataset, test_G, test_stats = test_instances[0]
    print(f"\nTest instance: {test_name} (n={test_stats['n']}, m={test_stats['m']})")
    det_solution = build_deterministic_dfs_mipstart(test_G)
    print(f"  Deterministic DFS height: {det_solution['H']}")
    rand_solution = build_randomized_dfs_mipstart(test_G, seed=42)
    print(f"  Randomized DFS height: {rand_solution['H']}")
    multi_solution = build_multiple_mipstarts(test_G, num_runs=5)
    print(f"  Best height from multiple randomized runs: {multi_solution['H']}")
    nodes = list(test_G.nodes())
    if nodes:
        depths = [det_solution['depth'][v] for v in nodes if det_solution['depth'][v] is not None]
        roots = [v for v in nodes if det_solution['root'][v] == 1]
        print(f"  Solution check: depth range=[{min(depths) if depths else 0}, {max(depths) if depths else 0}], number of roots={len(roots)}")

print("\nTesting solver configuration...")
test_configs = [
    {"name": "Base config", "mipstart_enabled": True, "lazy_constraints_enabled": True, "gurobi_presolve": 2},
    {"name": "No MIPStart", "mipstart_enabled": False, "lazy_constraints_enabled": True, "gurobi_presolve": 2},
    {"name": "No Lazy constraints", "mipstart_enabled": True, "lazy_constraints_enabled": False, "gurobi_presolve": 2},
    {"name": "No Presolve", "mipstart_enabled": True, "lazy_constraints_enabled": True, "gurobi_presolve": 0},
]

for config in test_configs:
    print(f"  {config['name']}: MIP={config['mipstart_enabled']}, Lazy={config['lazy_constraints_enabled']}, Presolve={config['gurobi_presolve']}")

print("\nMIPStart and solver configuration tests complete")


Testing MIPStart generation...

Test instance: Brinkmann (n=21, m=42)
  Deterministic DFS height: 20
  Randomized DFS height: 20
  Best height from multiple randomized runs: 18
  Solution check: depth range=[1, 20], number of roots=1

Testing solver configuration...
  Base config: MIP=True, Lazy=True, Presolve=2
  No MIPStart: MIP=False, Lazy=True, Presolve=2
  No Lazy constraints: MIP=True, Lazy=False, Presolve=2
  No Presolve: MIP=True, Lazy=True, Presolve=0

MIPStart and solver configuration tests complete


In [33]:
def solve_single_component_ilp(G: nx.Graph, 
                              component_config: Dict[str, Any],
                              encoding_variant: str = "v1_baseline",
                              mipstart_runs: int = 10) -> Tuple[Optional[int], bool, Dict[str, Any]]:
    n = G.number_of_nodes()
    if n <= 1:
        return n, False, {"variables": 0, "constraints": 0, "nodes": 0, "gap": 0.0}

    nodes = list(G.nodes())
    edges = list(G.edges())
    LB, UB = estimate_bounds(G)
    U = UB if UB > 0 else n

    model = gp.Model(f"Treedepth_ILP_{encoding_variant}")
    setup_gurobi_model(model, component_config)

    d = model.addVars(nodes, vtype=GRB.INTEGER, lb=1, ub=U, name="d")
    r = model.addVars(nodes, vtype=GRB.BINARY, name="r")
    p = model.addVars(nodes, nodes, vtype=GRB.BINARY, name="p")
    a = model.addVars(nodes, nodes, vtype=GRB.BINARY, name="a")

    try:
        model.setObjective(gp.max_([d[v] for v in nodes]), GRB.MINIMIZE)
        use_auxiliary_var = False
    except:
        max_depth = model.addVar(vtype=GRB.INTEGER, lb=LB, ub=UB, name="max_depth")
        model.setObjective(max_depth, GRB.MINIMIZE)
        for v in nodes:
            model.addConstr(max_depth >= d[v], name=f"max_depth_{v}")
        use_auxiliary_var = True

    for i in nodes:
        model.addConstr(p[i, i] == 0, name=f"no_self_parent_{i}")
        model.addConstr(a[i, i] == 0, name=f"no_self_ancestor_{i}")

    for v in nodes:
        model.addConstr(gp.quicksum(p[u, v] for u in nodes if u != v) + r[v] == 1, 
                       name=f"unique_parent_{v}")

    for v in nodes:
        model.addConstr(d[v] <= 1 + U * (1 - r[v]), name=f"root_depth_ub_{v}")
        model.addConstr(d[v] >= 1 - U * (1 - r[v]), name=f"root_depth_lb_{v}")

    for u in nodes:
        for v in nodes:
            if u != v:
                model.addConstr(d[v] - d[u] >= 1 - U * (1 - p[u, v]), 
                               name=f"parent_depth_lb_{u}_{v}")
                model.addConstr(d[v] - d[u] <= 1 + U * (1 - p[u, v]), 
                               name=f"parent_depth_ub_{u}_{v}")

    for u in nodes:
        for v in nodes:
            if u != v:
                model.addConstr(d[u] + 1 <= d[v] + U * (1 - a[u, v]), 
                               name=f"ancestor_depth_{u}_{v}")

    for v in nodes:
        model.addConstr(gp.quicksum(a[u, v] for u in nodes if u != v) == d[v] - 1, 
                       name=f"ancestor_count_{v}")

    for u in nodes:
        for v in nodes:
            if u != v:
                model.addConstr(a[u, v] >= p[u, v], name=f"ancestor_includes_parent_{u}_{v}")

    if encoding_variant in ["v2_explicit_transitivity", "v4_with_symmetry", "v5_enhanced_constraints"]:
        transitivity_count = 0
        for i in nodes:
            for j in nodes:
                if i != j:
                    for k in nodes:
                        if k != i and k != j:
                            model.addConstr(a[i, k] >= a[i, j] + a[j, k] - 1, 
                                           name=f"transitivity_{i}_{j}_{k}")
                            transitivity_count += 1
                if transitivity_count >= n * n:
                    break
            if transitivity_count >= n * n:
                break

    if encoding_variant in ["v3_tight_bounds", "v5_enhanced_constraints"]:
        for v in nodes:
            neighbors = list(G.neighbors(v))
            if neighbors:
                model.addConstr(gp.quicksum(a[u, v] + a[v, u] for u in neighbors) >= 1,
                               name=f"tight_neighbor_{v}")

    if encoding_variant in ["v4_with_symmetry", "v5_enhanced_constraints"]:
        for i in nodes:
            for j in nodes:
                if i < j:
                    model.addConstr(a[i, j] + a[j, i] <= 1, name=f"symmetry_break_{i}_{j}")

    for u, v in edges:
        model.addConstr(a[u, v] + a[v, u] >= 1, name=f"edge_constraint_{u}_{v}")

    solve_stats = {"variables": 0, "constraints": 0, "nodes": 0, "gap": 0.0}
    
    if component_config.get("mipstart_enabled", True):
        try:
            ms = build_multiple_mipstarts(G, EXPERIMENTAL_CONFIG['MIPSTART_RUNS'])
            if not use_auxiliary_var:
                pass
            else:
                max_depth.Start = ms["H"]
            for v in nodes:
                d[v].Start = ms["depth"][v]
                r[v].Start = ms["root"][v]
                for u in nodes:
                    if u != v:
                        p[u, v].Start = 1 if ms["parent"][v] == u else 0
                        a[u, v].Start = ms["ancestor"][(u, v)]
        except Exception as e:
            print(f"    Warning: Failed to set MIPStart: {e}")

    if component_config.get("lazy_constraints_enabled", True):
        model._a = a
        model._nodes = nodes
        callback_func = lambda m, w: lazy_transitivity_callback(m, w)
    else:
        callback_func = None
        print(f"    Adding explicit transitivity constraints...", end="")
        transitivity_constraints_added = 0
        for i in nodes:
            for j in nodes:
                if i != j:
                    for k in nodes:
                        if k != i and k != j:
                            model.addConstr(a[i, k] >= a[i, j] + a[j, k] - 1, 
                                           name=f"explicit_transitivity_{i}_{j}_{k}")
                            transitivity_constraints_added += 1
        print(f" {transitivity_constraints_added}")

    try:
        if callback_func:
            model.optimize(callback_func)
        else:
            model.optimize()
        solve_stats = {
            "variables": model.NumVars,
            "constraints": model.NumConstrs,
            "nodes": int(model.NodeCount) if hasattr(model, 'NodeCount') else 0,
            "gap": model.MIPGap if hasattr(model, 'MIPGap') else 0.0,
            "obj_val": model.ObjVal if model.Status == GRB.OPTIMAL else None,
            "sol_count": model.SolCount if hasattr(model, 'SolCount') else 0
        }
        if model.Status == GRB.OPTIMAL:
            result = int(max(d[v].X for v in nodes))
            return result, False, solve_stats
        elif model.Status == GRB.TIME_LIMIT and model.SolCount > 0:
            result = int(max(d[v].X for v in nodes))
            return result, True, solve_stats
        else:
            return None, True, solve_stats
    except Exception as e:
        print(f"    Solve exception: {e}")
        return None, True, solve_stats

def solve_treedepth_ilp(G: nx.Graph,
                       component_config: Dict[str, Any],
                       encoding_variant: str = "v1_baseline",
                       enable_preprocessing: bool = True,
                       mipstart_runs: int = 10) -> Tuple[Optional[int], bool, Dict[str, Any]]:
    start_time = time.time()
    processed_G, apex_buffer, preprocess_stats = preprocess_graph(G, enable_preprocessing)
    if processed_G.number_of_nodes() <= 1:
        solve_time = time.time() - start_time
        stats = {
            **preprocess_stats,
            "solve_time": solve_time,
            "components_solved": 1,
            "total_variables": 0,
            "total_constraints": 0,
            "max_nodes": 0,
            "avg_gap": 0.0
        }
        return processed_G.number_of_nodes() + apex_buffer, False, stats

    max_td = 0
    overall_timeout = False
    total_vars = 0
    total_constrs = 0
    total_nodes = 0
    gaps = []
    components_count = 0
    
    for comp in nx.connected_components(processed_G):
        sub_G = processed_G.subgraph(comp).copy()
        sub_G = nx.convert_node_labels_to_integers(sub_G, first_label=0, ordering="default")
        td, timeout, comp_stats = solve_single_component_ilp(
            sub_G, component_config, encoding_variant, mipstart_runs
        )
        if td is None:
            solve_time = time.time() - start_time
            stats = {
                **preprocess_stats,
                "solve_time": solve_time,
                "components_solved": components_count,
                "total_variables": total_vars,
                "total_constraints": total_constrs,
                "max_nodes": total_nodes,
                "avg_gap": np.mean(gaps) if gaps else 0.0
            }
            return None, True, stats
        max_td = max(max_td, td)
        if timeout:
            overall_timeout = True
        total_vars += comp_stats["variables"]
        total_constrs += comp_stats["constraints"]
        total_nodes += comp_stats["nodes"]
        if comp_stats["gap"] > 0:
            gaps.append(comp_stats["gap"])
        components_count += 1
    
    final_td = max_td + apex_buffer
    solve_time = time.time() - start_time
    final_stats = {
        **preprocess_stats,
        "solve_time": solve_time,
        "components_solved": components_count,
        "total_variables": total_vars,
        "total_constraints": total_constrs,
        "max_nodes": total_nodes,
        "avg_gap": np.mean(gaps) if gaps else 0.0
    }
    return final_td, overall_timeout, final_stats

print("Testing ILP solver...")

test_config = {
    "mipstart_enabled": True,
    "lazy_constraints_enabled": True,
    "gurobi_presolve": 2
}

small_instances = [(name, dataset, G, stats) for name, dataset, G, stats in test_instances if stats["n"] <= 8]

if small_instances:
    test_name, test_dataset, test_G, test_stats = small_instances[0]
    print(f"\nTest instance: {test_name} (n={test_stats['n']}, m={test_stats['m']})")
    result, timeout, stats = solve_treedepth_ilp(
        test_G, test_config, "v1_baseline", enable_preprocessing=True, mipstart_runs=3
    )
    if result is not None:
        print(f"  Result: treedepth = {result}")
        print(f"  Timeout: {timeout}")
        print(f"  Solve time: {stats['solve_time']:.3f}s")
        print(f"  Variables: {stats['total_variables']}")
        print(f"  Constraints: {stats['total_constraints']}")
        print(f"  Preprocessing reduction: {stats['reduction_rate']:.1%}")
    else:
        print("  Solve failed")

print("\n ILP core solver tests complete")


Testing ILP solver...

Test instance: Bull (n=5, m=5)
  Result: treedepth = 3
  Timeout: False
  Solve time: 0.005s
  Variables: 61
  Constraints: 120
  Preprocessing reduction: 0.0%

 ILP core solver tests complete


In [ ]:
import subprocess
import tempfile
import shutil
import threading
import signal
import gc
import os
import time
import numpy as np
import pandas as pd
import networkx as nx

EXPERIMENT_4_6_CONFIG = {
    "max_nodes": 50,
    "max_edges": 10000,
    "min_nodes": 3,
    "timeout_per_instance": 1200,
    "buffer_time": 10,
    "skip_large_instances": True,
    "enable_progress_display": True,
    "save_intermediate_results": False,
    "max_density": 1.0,
    "min_density": 0.0,
}

print("=" * 60)
print("Experiment 4.6: Optimal ILP vs SAT performance comparison (unified config)")
print("=" * 60)
print("Parameters:")
for key, value in EXPERIMENT_4_6_CONFIG.items():
    print(f"  {key}: {value}")
print()

def filter_instances_by_config(test_instances, config):
    selected_instances = []
    skipped_stats = {
        "too_many_nodes": 0,
        "too_many_edges": 0, 
        "too_few_nodes": 0,
        "density_out_of_range": 0,
        "large_instance_skip": 0
    }
    for name, dataset, G, stats in test_instances:
        if stats["n"] > config["max_nodes"]:
            skipped_stats["too_many_nodes"] += 1
            continue
        if stats["n"] < config["min_nodes"]:
            skipped_stats["too_few_nodes"] += 1
            continue
        if stats["m"] > config["max_edges"]:
            skipped_stats["too_many_edges"] += 1
            continue
        density = stats["density"]
        if density > config["max_density"] or density < config["min_density"]:
            skipped_stats["density_out_of_range"] += 1
            continue
        if config["skip_large_instances"] and stats["n"] > 30:
            skipped_stats["large_instance_skip"] += 1
            continue
        selected_instances.append((name, dataset, G, stats))
    return selected_instances, skipped_stats

def solve_with_strict_timeout(solve_func, timeout, *args, **kwargs):
    result = {"td": None, "timeout": True, "stats": {}, "exception": None}
    def target():
        try:
            result["td"], result["timeout"], result["stats"] = solve_func(*args, **kwargs)
        except Exception as e:
            result["exception"] = e
    thread = threading.Thread(target=target)
    thread.daemon = True
    thread.start()
    thread.join(timeout)
    if thread.is_alive():
        result["timeout"] = True
    return result["td"], result["timeout"], result["stats"]

def prepare_sat_solver():
    print("Checking SAT solver environment...")
    try:
        result = subprocess.run(['glucose', '-h'], capture_output=True, text=True, timeout=5)
        print("Glucose SAT solver available")
        return True
    except (FileNotFoundError, subprocess.TimeoutExpired):
        print("Glucose solver not found, skipping SAT comparison")
        return False

def make_vars(g, width):
    nv = g.number_of_nodes()
    p = [[[0]*width for _ in range(nv)] for __ in range(nv)]
    cur = 1
    for u in range(nv):
        for v in range(u, nv):
            for i in range(width):
                p[u][v][i] = cur
                cur += 1
    return p, cur-1

def generate_sat_encoding(g, width):
    s, nvar = make_vars(g, width)
    clauses = []
    nv = g.number_of_nodes()
    nclauses = 0
    for u in range(nv):
        for v in range(u, nv):
            clauses.append(f"{s[u][v][width-1]} 0")
            nclauses += 1
            clauses.append(f"-{s[u][v][0]} 0")
            nclauses += 1
    for u in range(nv):
        for v in range(u, nv):
            for i in range(1, width):
                clauses.append(f"-{s[u][v][i-1]} {s[u][v][i]} 0")
                nclauses += 1
    for u in range(nv):
        for v in range(u+1, nv):
            for w in range(v+1, nv):
                for i in range(width):
                    clauses.append(f"-{s[u][v][i]} -{s[u][w][i]} {s[v][w][i]} 0")
                    clauses.append(f"-{s[u][v][i]} -{s[v][w][i]} {s[u][w][i]} 0")
                    clauses.append(f"-{s[u][w][i]} -{s[v][w][i]} {s[u][v][i]} 0")
                    nclauses += 3
    for u in range(nv):
        for v in range(u+1, nv):
            for i in range(width):
                clauses.append(f"-{s[u][v][i]} {s[u][u][i]} 0")
                clauses.append(f"-{s[u][v][i]} {s[v][v][i]} 0")
                nclauses += 2
    for u in range(nv):
        for v in range(u+1, nv):
            for i in range(1, width):
                clauses.append(f"-{s[u][v][i]} {s[u][u][i-1]} {s[v][v][i-1]} 0")
                nclauses += 1
    for (u, v) in g.edges():
        if u > v:
            u, v = v, u
        for i in range(1, width):
            clauses.append(f"-{s[u][u][i]} {s[u][u][i-1]} -{s[v][v][i]} {s[u][v][i]} 0")
            clauses.append(f"-{s[u][u][i]} {s[v][v][i-1]} -{s[v][v][i]} {s[u][v][i]} 0")
            nclauses += 2
    preamble = f"p cnf {nvar} {nclauses}"
    return preamble + "\n" + "\n".join(clauses) + "\n"

def solve_component_sat_with_timeout(g, timeout=120, temp_dir=None):
    if temp_dir is None:
        temp_dir = tempfile.mkdtemp()
    start_time = time.time()
    encoding_times = []
    solving_times = []
    n = g.number_of_nodes()
    if n <= 1:
        return n, False, encoding_times, solving_times, 0, 0
    timeout_occurred = False
    variables_generated = 0
    clauses_generated = 0
    for w in range(n+1, 1, -1):
        elapsed_time = time.time() - start_time
        if elapsed_time >= timeout:
            timeout_occurred = True
            break
        encoding_start = time.time()
        cnf_str = generate_sat_encoding(g, w)
        encoding_time = time.time() - encoding_start
        encoding_times.append(encoding_time)
        lines = cnf_str.strip().split('\n')
        header = lines[0]
        parts = header.split()
        if len(parts) >= 4:
            variables_generated = int(parts[2])
            clauses_generated = int(parts[3])
        cnf_file = os.path.join(temp_dir, f'temp_{w}.cnf')
        with open(cnf_file, 'w') as f:
            f.write(cnf_str)
        remaining_time = timeout - elapsed_time
        if remaining_time <= 0:
            timeout_occurred = True
            break
        sol_file = os.path.join(temp_dir, f'temp_{w}.sol')
        cmd = ['glucose', f'-cpu-lim={int(remaining_time)}', cnf_file, sol_file]
        solving_start = time.time()
        try:
            result = subprocess.run(cmd, capture_output=True, text=True, 
                                  timeout=remaining_time+5)
            solving_time = time.time() - solving_start
            solving_times.append(solving_time)
            rc = result.returncode
            if rc == 0:
                timeout_occurred = True
            elif rc == 20:
                try:
                    os.remove(cnf_file)
                    if os.path.exists(sol_file):
                        os.remove(sol_file)
                except:
                    pass
                return w, timeout_occurred, encoding_times, solving_times, variables_generated, clauses_generated
        except subprocess.TimeoutExpired:
            solving_time = time.time() - solving_start
            solving_times.append(solving_time)
            timeout_occurred = True
        try:
            os.remove(cnf_file)
            if os.path.exists(sol_file):
                os.remove(sol_file)
        except:
            pass
    return 1, timeout_occurred, encoding_times, solving_times, variables_generated, clauses_generated

def solve_treedepth_sat_with_strict_timeout(G, timeout=120, enable_preprocessing=True):
    start_time = time.time()
    processed_G, apex_buffer, preprocess_stats = preprocess_graph(G, enable_preprocessing)
    if processed_G.number_of_nodes() <= 1:
        solve_time = time.time() - start_time
        stats = {
            **preprocess_stats,
            "solve_time": solve_time,
            "components_solved": 1,
            "total_encoding_time": 0,
            "total_solving_time": 0,
            "sat_variables": 0,
            "sat_clauses": 0
        }
        return processed_G.number_of_nodes() + apex_buffer, False, stats
    temp_dir = tempfile.mkdtemp()
    try:
        max_td = 0
        overall_timeout = False
        total_encoding_time = 0
        total_solving_time = 0
        max_variables = 0
        max_clauses = 0
        components_count = 0
        for comp in nx.connected_components(processed_G):
            elapsed_time = time.time() - start_time
            if elapsed_time >= timeout:
                overall_timeout = True
                break
            sub_G = processed_G.subgraph(comp).copy()
            sub_G = nx.convert_node_labels_to_integers(sub_G, first_label=0, ordering="default")
            remaining_timeout = timeout - elapsed_time
            td, comp_timeout, enc_times, sol_times, variables, clauses = solve_component_sat_with_timeout(
                sub_G, remaining_timeout, temp_dir
            )
            if td is None:
                solve_time = time.time() - start_time
                stats = {
                    **preprocess_stats,
                    "solve_time": solve_time,
                    "components_solved": components_count,
                    "total_encoding_time": total_encoding_time,
                    "total_solving_time": total_solving_time,
                    "sat_variables": max_variables,
                    "sat_clauses": max_clauses
                }
                return None, True, stats
            max_td = max(max_td, td)
            if comp_timeout:
                overall_timeout = True
            total_encoding_time += sum(enc_times)
            total_solving_time += sum(sol_times)
            max_variables = max(max_variables, variables)
            max_clauses = max(max_clauses, clauses)
            components_count += 1
        final_solve_time = time.time() - start_time
        if final_solve_time >= timeout:
            overall_timeout = True
        final_td = max_td + apex_buffer
        final_stats = {
            **preprocess_stats,
            "solve_time": final_solve_time,
            "components_solved": components_count,
            "total_encoding_time": total_encoding_time,
            "total_solving_time": total_solving_time,
            "sat_variables": max_variables,
            "sat_clauses": max_clauses
        }
        return final_td, overall_timeout, final_stats
    finally:
        try:
            shutil.rmtree(temp_dir)
        except:
            pass

def run_ilp_vs_sat_experiment_with_unified_config():
    global OPTIMAL_ILP_CONFIG
    OPTIMAL_ILP_CONFIG = {
        'config_name': 'presolve_only',
        'config_description': 'Only Gurobi presolve - from Experiment 4.4',
        'accuracy_rate': 0.771,
        'encoding_variant': 'v5_enhanced_constraints',
        'preprocessing_enabled': False,
        'mipstart_enabled': False,
        'lazy_constraints_enabled': False,
        'gurobi_presolve': 2
    }
    print("Using hardcoded best result from Experiment 4.5:")
    print(f"  Best encoding variant: {OPTIMAL_ILP_CONFIG['encoding_variant']}")
    print(f"  Accuracy: {OPTIMAL_ILP_CONFIG['accuracy_rate']:.1%}")
    print(f"  Config name: {OPTIMAL_ILP_CONFIG['config_name']}")
    print(f"  Preprocessing: {OPTIMAL_ILP_CONFIG['preprocessing_enabled']}")
    print(f"  MIPStart: {OPTIMAL_ILP_CONFIG['mipstart_enabled']}")
    print(f"  Lazy constraints: {OPTIMAL_ILP_CONFIG['lazy_constraints_enabled']}")
    print(f"  Gurobi presolve: {OPTIMAL_ILP_CONFIG['gurobi_presolve']}")
    print()
    config = EXPERIMENT_4_6_CONFIG
    print("Running with unified configuration:")
    print(f"  Time limit: {config['timeout_per_instance']}s ({config['timeout_per_instance']/60:.1f}min)")
    print(f"  Node limit: {config['min_nodes']}-{config['max_nodes']}")
    print(f"  Edge limit: ≤{config['max_edges']}")
    print(f"  Density range: [{config['min_density']:.2f}, {config['max_density']:.2f}]")
    print(f"  Golden-standard instances: {len(GOLDEN_STANDARD) if 'GOLDEN_STANDARD' in globals() else 'undefined'}")
    print()
    optimal_ilp_config = OPTIMAL_ILP_CONFIG.copy()
    print("Optimal ILP configuration (from Experiment 4.5 hardcoded result):")
    print(f"  Encoding variant: {optimal_ilp_config['encoding_variant']}")
    print(f"  Accuracy: {optimal_ilp_config['accuracy_rate']:.1%}")
    print(f"  Preprocessing: {optimal_ilp_config['preprocessing_enabled']}")
    print()
    sat_available = prepare_sat_solver()
    if not sat_available:
        print("SAT solver unavailable; running ILP only...")
        return None, None
    try:
        selected_instances, skipped_stats = filter_instances_by_config(test_instances, config)
    except NameError:
        print("Error: test_instances is not defined; please ensure test instances are loaded")
        return None, None
    print("Instance filtering result:")
    print(f"  Original instances: {len(test_instances)}")
    print(f"  Selected instances: {len(selected_instances)}")
    print("  Skip stats:")
    for reason, count in skipped_stats.items():
        if count > 0:
            reason_desc = {
                "too_many_nodes": f"nodes > {config['max_nodes']}",
                "too_many_edges": f"edges > {config['max_edges']}",
                "too_few_nodes": f"nodes < {config['min_nodes']}",
                "density_out_of_range": "density out of range",
                "large_instance_skip": "large instance skipped"
            }
            print(f"    {reason_desc[reason]}: {count}")
    def categorize_instance_size(n):
        if n <= 10:
            return "TINY"
        elif n <= 20:
            return "SMALL" 
        elif n <= 30:
            return "MEDIUM"
        else:
            return "LARGE"
    size_categories = {"TINY": [], "SMALL": [], "MEDIUM": [], "LARGE": []}
    for name, dataset, G, stats in selected_instances:
        category = categorize_instance_size(stats["n"])
        size_categories[category].append((name, dataset, G, stats))
    print("\nSize distribution of selected instances:")
    for category, instances in size_categories.items():
        if instances:
            size_range = {"TINY": "≤10", "SMALL": "11-20", "MEDIUM": "21-30", "LARGE": "31-50"}[category]
            print(f"  {category} ({size_range} nodes): {len(instances)} instances")
    total_selected = len(selected_instances)
    estimated_max_time = total_selected * 2 * config['timeout_per_instance']
    print("\nTime estimate:")
    print(f"  Maximum possible runtime: {estimated_max_time/3600:.1f} hours")
    print(f"  Expected actual runtime: {estimated_max_time*0.4/3600:.1f} hours (rough 40% estimate)")
    print()
    results = []
    method_performance = {
        "ILP": {"optimal": 0, "timeout": 0, "failed": 0, "times": [], "success_times": []},
        "SAT": {"optimal": 0, "timeout": 0, "failed": 0, "times": [], "success_times": []}
    }
    method_validations = {"ILP": [], "SAT": []}
    ilp_solver_config = {
        "mipstart_enabled": optimal_ilp_config['mipstart_enabled'],
        "lazy_constraints_enabled": optimal_ilp_config['lazy_constraints_enabled'],
        "gurobi_presolve": optimal_ilp_config['gurobi_presolve']
    }
    print(f"Starting ILP vs SAT comparison test ({total_selected} selected instances)...")
    experiment_start_time = time.time()
    for inst_idx, (instance, dataset, G, graph_stats) in enumerate(selected_instances, 1):
        size_category = categorize_instance_size(graph_stats["n"])
        elapsed_time = time.time() - experiment_start_time
        if inst_idx > 1:
            avg_time_per_instance = elapsed_time / (inst_idx - 1)
            remaining_instances = total_selected - inst_idx + 1
            eta_seconds = remaining_instances * avg_time_per_instance
            eta_str = f", ETA: {eta_seconds/3600:.1f}h" if eta_seconds > 3600 else f", ETA: {eta_seconds/60:.0f}min"
        else:
            eta_str = ""
        print(f"\n[{inst_idx}/{total_selected}] Testing instance: {dataset}/{instance} ({size_category}){eta_str}")
        print(f"  Graph size: n={graph_stats['n']}, m={graph_stats['m']}, density={graph_stats['density']:.3f}")
        print(f"  Elapsed: {elapsed_time/60:.1f} min")
        print("    [ILP]", end=" ")
        ilp_start_time = time.time()
        try:
            original_timeout = EXPERIMENTAL_CONFIG['TIMEOUT']
            EXPERIMENTAL_CONFIG['TIMEOUT'] = config['timeout_per_instance'] - config['buffer_time']
            td_ilp, timeout_ilp_internal, stats_ilp = solve_treedepth_ilp(
                G, ilp_solver_config, optimal_ilp_config['encoding_variant'],
                enable_preprocessing=optimal_ilp_config['preprocessing_enabled'],
                mipstart_runs=EXPERIMENTAL_CONFIG['MIPSTART_RUNS']
            )
            EXPERIMENTAL_CONFIG['TIMEOUT'] = original_timeout
            time_ilp = time.time() - ilp_start_time
            if time_ilp >= config['timeout_per_instance']:
                timeout_ilp = True
                if td_ilp is not None:
                    status_ilp = "timeout"
                else:
                    status_ilp = "failed"
            else:
                timeout_ilp = timeout_ilp_internal
                if td_ilp is not None:
                    if timeout_ilp:
                        status_ilp = "timeout"
                    else:
                        status_ilp = "optimal"
                else:
                    status_ilp = "failed"
            if status_ilp == "optimal":
                method_performance["ILP"]["optimal"] += 1
                method_performance["ILP"]["success_times"].append(time_ilp)
                print(f"treedepth = {td_ilp}, {time_ilp:.2f}s (optimal)", end="")
            elif status_ilp == "timeout":
                method_performance["ILP"]["timeout"] += 1
                print(f"treedepth ≤ {td_ilp if td_ilp else '?'}, {time_ilp:.2f}s (timeout)", end="")
            else:
                method_performance["ILP"]["failed"] += 1
                print(f"failed, {time_ilp:.2f}s", end="")
            method_performance["ILP"]["times"].append(time_ilp)
        except Exception as e:
            time_ilp = time.time() - ilp_start_time
            td_ilp, timeout_ilp, status_ilp = None, True, "failed"
            stats_ilp = {"total_variables": 0, "total_constraints": 0}
            method_performance["ILP"]["failed"] += 1
            method_performance["ILP"]["times"].append(time_ilp)
            print(f"Exception: {str(e)[:30]}", end="")
            EXPERIMENTAL_CONFIG['TIMEOUT'] = original_timeout
        validation_ilp = validate_result(instance, td_ilp, timeout_ilp)
        method_validations["ILP"].append(validation_ilp)
        if validation_ilp["validation_status"] != "no_golden_standard":
            if validation_ilp["is_correct"]:
                print(" ✅")
            else:
                if validation_ilp["error_type"] == "timeout_not_optimal":
                    print("(timeout)")
                else:
                    print(f"({validation_ilp['error_type']})")
        else:
            print("no")
        print("    [SAT]", end=" ")
        sat_start_time = time.time()
        try:
            td_sat, timeout_sat, stats_sat = solve_treedepth_sat_with_strict_timeout(
                G, timeout=config['timeout_per_instance'],
                enable_preprocessing=optimal_ilp_config['preprocessing_enabled']
            )
            time_sat = time.time() - sat_start_time
            if time_sat >= config['timeout_per_instance']:
                timeout_sat = True
            if td_sat is not None:
                if timeout_sat:
                    status_sat = "timeout"
                    method_performance["SAT"]["timeout"] += 1
                    print(f"treedepth ≤ {td_sat}, {time_sat:.2f}s (timeout)", end="")
                else:
                    status_sat = "optimal"
                    method_performance["SAT"]["optimal"] += 1
                    method_performance["SAT"]["success_times"].append(time_sat)
                    print(f"treedepth = {td_sat}, {time_sat:.2f}s (optimal)", end="")
            else:
                status_sat = "failed"
                method_performance["SAT"]["failed"] += 1
                print(f"failed, {time_sat:.2f}s", end="")
            method_performance["SAT"]["times"].append(time_sat)
        except Exception as e:
            time_sat = time.time() - sat_start_time
            td_sat, timeout_sat, status_sat = None, True, "failed"
            stats_sat = {"sat_variables": 0, "sat_clauses": 0}
            method_performance["SAT"]["failed"] += 1
            method_performance["SAT"]["times"].append(time_sat)
            print(f"Exception: {str(e)[:30]}", end="")
        validation_sat = validate_result(instance, td_sat, timeout_sat)
        method_validations["SAT"].append(validation_sat)
        if validation_sat["validation_status"] != "no_golden_standard":
            if validation_sat["is_correct"]:
                print("OK")
            else:
                if validation_sat["error_type"] == "timeout_not_optimal":
                    print("(timeout)")
                else:
                    print(f"({validation_sat['error_type']})")
        else:
            print("no")
        consistency_check = "unknown"
        if (td_ilp is not None and td_sat is not None and 
            status_ilp in ["optimal", "timeout"] and status_sat in ["optimal", "timeout"]):
            consistency_check = "consistent" if td_ilp == td_sat else "inconsistent"
            if consistency_check == "inconsistent":
                print(f"Inconsistent results: ILP={td_ilp}, SAT={td_sat}")
        base_record = {
            "experiment": "E4_6_ilp_vs_sat_unified",
            "instance": instance,
            "dataset": dataset,
            "size_category": size_category,
            "n": graph_stats["n"],
            "m": graph_stats["m"],
            "graph_density": round(graph_stats["density"], 4),
            "graph_components": graph_stats["components"],
            "consistency_check": consistency_check,
            "timeout_used": config['timeout_per_instance'],
            "max_nodes_limit": config['max_nodes'],
            "max_edges_limit": config['max_edges'],
            "golden_standard": validation_ilp["golden_value"],
        }
        ilp_record = base_record.copy()
        ilp_record.update({
            "method": "ILP",
            "treedepth": td_ilp,
            "time": round(time_ilp, 3),
            "status": status_ilp,
            "timeout": timeout_ilp,
            "preprocessing_enabled": optimal_ilp_config['preprocessing_enabled'],
            "encoding_variant": optimal_ilp_config['encoding_variant'],
            "mipstart_enabled": optimal_ilp_config['mipstart_enabled'],
            "lazy_constraints_enabled": optimal_ilp_config['lazy_constraints_enabled'],
            "gurobi_presolve": optimal_ilp_config['gurobi_presolve'],
            "problem_variables": stats_ilp.get("total_variables", 0),
            "problem_constraints": stats_ilp.get("total_constraints", 0),
            "gurobi_objval": td_ilp,
            "gurobi_mip_gap": 0.0,
            "gurobi_node_count": 0,
            "preprocessing_reduction": stats_ilp.get("reduction_rate", 0.0),
            "memory_usage_mb": 0.0,
            "is_correct": validation_ilp["is_correct"],
            "validation_status": validation_ilp["validation_status"],
            "error_type": validation_ilp["error_type"]
        })
        results.append(ilp_record)
        sat_record = base_record.copy()
        sat_record.update({
            "method": "SAT",
            "treedepth": td_sat,
            "time": round(time_sat, 3),
            "status": status_sat,
            "timeout": timeout_sat,
            "preprocessing_enabled": optimal_ilp_config['preprocessing_enabled'],
            "encoding_variant": "sat_original_paper",
            "mipstart_enabled": False,
            "lazy_constraints_enabled": False,
            "gurobi_presolve": 0,
            "problem_variables": stats_sat.get("sat_variables", 0),
            "problem_constraints": stats_sat.get("sat_clauses", 0),
            "gurobi_objval": td_sat,
            "gurobi_mip_gap": 0.0,
            "gurobi_node_count": 0,
            "preprocessing_reduction": stats_sat.get("reduction_rate", 0.0),
            "memory_usage_mb": 0.0,
            "is_correct": validation_sat["is_correct"],
            "validation_status": validation_sat["validation_status"],
            "error_type": validation_sat["error_type"]
        })
        results.append(sat_record)
        gc.collect()
        if config["enable_progress_display"] and (inst_idx % 3 == 0 or inst_idx == total_selected):
            ilp_success = method_performance["ILP"]["optimal"] + method_performance["ILP"]["timeout"]
            sat_success = method_performance["SAT"]["optimal"] + method_performance["SAT"]["timeout"]
            ilp_analysis = analyze_validation_results(method_validations["ILP"])
            sat_analysis = analyze_validation_results(method_validations["SAT"])
            print("Progress summary:")
            print(f"      Completed: {inst_idx}/{total_selected} ({inst_idx/total_selected*100:.1f}%)")
            print(f"      ILP: Success {ilp_success}/{inst_idx} ({ilp_success/inst_idx*100:.1f}%), Accuracy {ilp_analysis.get('accuracy_rate', 0):.1%}")
            print(f"      SAT: Success {sat_success}/{inst_idx} ({sat_success/inst_idx*100:.1f}%), Accuracy {sat_analysis.get('accuracy_rate', 0):.1%}")
            print(f"      Total elapsed: {elapsed_time/60:.1f} min")
    total_experiment_time = time.time() - experiment_start_time
    print(f"\nTotal experiment runtime: {total_experiment_time/3600:.2f} hours")
    output_file = OUTPUT_FILES["final_comparison"]
    df_results = pd.DataFrame(results)
    df_results.to_csv(output_file, index=False, encoding='utf-8')
    print("\n" + "=" * 60)
    print("Experiment 4.6 - Optimal ILP vs SAT performance comparison (unified parameters)")
    print("=" * 60)
    print("Experiment configuration summary:")
    print(f"  Number of test instances: {total_selected}")
    print(f"  Unified time limit: {config['timeout_per_instance']}s ({config['timeout_per_instance']/60:.1f}min)")
    print(f"  Node limit: {config['min_nodes']}-{config['max_nodes']}")
    print(f"  Edge limit: ≤{config['max_edges']}")
    print(f"  Total runtime: {total_experiment_time/3600:.2f} hours")
    ilp_results = [r for r in results if r["method"] == "ILP"]
    sat_results = [r for r in results if r["method"] == "SAT"]
    ilp_metrics = calculate_method_metrics(ilp_results, "ILP")
    sat_metrics = calculate_method_metrics(sat_results, "SAT")
    print(f"\n Accuracy and performance comparison (unified time limit {config['timeout_per_instance']}s):")
    print(f"{'Method':<8} {'Accuracy':<10} {'Optimal':<10} {'Success':<10} {'Correct/Test':<12} {'Avg time':<12} {'Grade':<10}")
    print("-" * 90)
    for metrics in [ilp_metrics, sat_metrics]:
        name = metrics["method_name"]
        accuracy = metrics["accuracy_rate"]
        optimal_rate = metrics.get("optimal_rate", 0)
        success_rate = metrics["success_rate"]
        correct = metrics["correct_instances"]
        testable = metrics["testable_instances"]
        avg_time = metrics["avg_time"]
        time_str = f"{avg_time:.2f}s" if avg_time != float('inf') else "∞"
        if accuracy >= 0.95:
            quality = "A+"
        elif accuracy >= 0.90:
            quality = "A"
        elif accuracy >= 0.80:
            quality = "B"
        elif accuracy >= 0.70:
            quality = "C"
        elif accuracy >= 0.50:
            quality = "D"
        else:
            quality = "F"
        print(f"{name:<8} {accuracy:>8.1%} {optimal_rate:>8.1%} {success_rate:>8.1%} {correct:>4}/{testable:<6} {time_str:>10} {quality:>8}")
    print("\n Method selection (based on accuracy):")
    accuracy_diff = ilp_metrics["accuracy_rate"] - sat_metrics["accuracy_rate"]
    if abs(accuracy_diff) < 0.02:
        print(f"  Accuracies are close (difference {abs(accuracy_diff):.1%} < 2%)")
        optimal_diff = ilp_metrics.get("optimal_rate", 0) - sat_metrics.get("optimal_rate", 0)
        if abs(optimal_diff) > 0.05:
            better_method = "ILP" if optimal_diff > 0 else "SAT"
            print(f"  Choose {better_method} (higher optimal rate: {optimal_diff:+.1%})")
            winner = better_method
            reason = f"Similar accuracy; {better_method} has higher optimal rate"
        else:
            time_diff = sat_metrics["avg_time"] - ilp_metrics["avg_time"]
            if abs(time_diff) > 10:
                better_method = "ILP" if time_diff > 0 else "SAT"
                print(f"  Choose {better_method} (better time performance)")
                winner = better_method
                reason = "Similar accuracy and optimal rate; better time performance"
            else:
                winner = "ILP"
                reason = "Similar across metrics; choose the more mature ILP method"
                print("  Choose ILP (metrics similar; choose the more mature method)")
    else:
        winner = "ILP" if accuracy_diff > 0 else "SAT"
        reason = f"Higher accuracy for {winner} ({accuracy_diff:+.1%})"
        print(f"  Choose {winner} (higher accuracy: {accuracy_diff:+.1%})")
    print("\n Detailed performance analysis:")
    print(f"\nTiming statistics (unified limit {config['timeout_per_instance']}s):")
    ilp_times = [r["time"] for r in ilp_results if r["status"] in ["optimal", "timeout"]]
    sat_times = [r["time"] for r in sat_results if r["status"] in ["optimal", "timeout"]]
    if ilp_times and sat_times:
        print(f"  ILP: mean {np.mean(ilp_times):.2f}s, median {np.median(ilp_times):.2f}s, max {np.max(ilp_times):.2f}s")
        print(f"  SAT: mean {np.mean(sat_times):.2f}s, median {np.median(sat_times):.2f}s, max {np.max(sat_times):.2f}s")
        time_ranges = [(0, 10), (10, 60), (60, 300), (300, config['timeout_per_instance'])]
        print("  Time distribution:")
        for start, end in time_ranges:
            ilp_count = len([t for t in ilp_times if start <= t < end])
            sat_count = len([t for t in sat_times if start <= t < end])
            range_desc = f"{start}-{end}s" if end < config['timeout_per_instance'] else f">{start}s"
            print(f"    {range_desc}: ILP {ilp_count}/{len(ilp_times)} ({ilp_count/len(ilp_times)*100:.1f}%), "
                    f"SAT {sat_count}/{len(sat_times)} ({sat_count/len(sat_times)*100:.1f}%)")
    print("\nBy-size analysis:")
    for category in ["TINY", "SMALL", "MEDIUM", "LARGE"]:
        if category in size_categories and size_categories[category]:
            instances_in_category = [r for r in results if r["size_category"] == category]
            if not instances_in_category:
                continue
            ilp_cat = [r for r in instances_in_category if r["method"] == "ILP"]
            sat_cat = [r for r in instances_in_category if r["method"] == "SAT"]
            if ilp_cat and sat_cat:
                ilp_cat_metrics = calculate_method_metrics(ilp_cat, f"ILP-{category}")
                sat_cat_metrics = calculate_method_metrics(sat_cat, f"SAT-{category}")
                size_range = {"TINY": "≤10", "SMALL": "11-20", "MEDIUM": "21-30", "LARGE": "31-50"}[category]
                print(f"  {category} ({size_range} nodes, {len(ilp_cat)} instances):")
                print(f"    ILP: accuracy {ilp_cat_metrics['accuracy_rate']:.1%}, optimal {ilp_cat_metrics.get('optimal_rate', 0):.1%}")
                print(f"    SAT: accuracy {sat_cat_metrics['accuracy_rate']:.1%}, optimal {sat_cat_metrics.get('optimal_rate', 0):.1%}")
    min_accuracy = min(ilp_metrics["accuracy_rate"], sat_metrics["accuracy_rate"])
    if min_accuracy < 0.5:
        print(f"\n Critical warning: best method accuracy is only {min_accuracy:.1%}")
        print("This suggests potential implementation issues, but the experiment continues")
    elif min_accuracy >= 0.95:
        print("\n Excellent: both methods have accuracy ≥95%")
    elif min_accuracy >= 0.8:
        print("\n✓ Good: both methods have accuracy ≥80%")
    print("\nConsistency analysis:")
    consistency_stats = {}
    for r in results[::2]:
        status = r["consistency_check"]
        consistency_stats[status] = consistency_stats.get(status, 0) + 1
    for status, count in consistency_stats.items():
        print(f"  {status}: {count} instances")
    print("\n Final recommendation:")
    print(f"  Recommended method: {winner}")
    print(f"  Reason: {reason}")
    print(f"  Method quality: {winner} accuracy is {(ilp_metrics if winner == 'ILP' else sat_metrics)['accuracy_rate']:.1%}")
    global FINAL_OPTIMAL_CONFIG
    FINAL_OPTIMAL_CONFIG = {
        **optimal_ilp_config,
        "recommended_method": winner,
        "recommendation_reason": reason,
        "ilp_accuracy": ilp_metrics["accuracy_rate"],
        "sat_accuracy": sat_metrics["accuracy_rate"],
        "ilp_optimal_rate": ilp_metrics.get("optimal_rate", 0),
        "sat_optimal_rate": sat_metrics.get("optimal_rate", 0),
        "ilp_success_rate": ilp_metrics["success_rate"],
        "sat_success_rate": sat_metrics["success_rate"],
        "ilp_avg_time": ilp_metrics["avg_time"],
        "sat_avg_time": sat_metrics["avg_time"],
        "instances_tested": total_selected,
        "experiment_duration_hours": total_experiment_time / 3600,
        "unified_timeout": config['timeout_per_instance'],
        "experiment_config": config.copy()
    }
    print(f"\nResults saved to: {output_file}")
    print(" Final method recommendation complete")
    return results, {"ILP": ilp_metrics, "SAT": sat_metrics}

print(" Notes on parameter tweaks:")
print("  To modify parameters, edit the EXPERIMENT_4_6_CONFIG dictionary above")
print("  Key parameters:")
print("    timeout_per_instance: time limit per instance (seconds)")
print("    max_nodes: maximum number of nodes")
print("    max_edges: maximum number of edges")
print("    skip_large_instances: whether to skip large instances")
print()

print("Running Experiment 4.6 with unified configuration (accuracy first)...")

final_comparison_results, final_methods_comparison = run_ilp_vs_sat_experiment_with_unified_config()

if final_comparison_results:
    print("\n Experiment 4.6 complete!")
    print(f" Recommended method: {FINAL_OPTIMAL_CONFIG['recommended_method']}")
    print(f" Recommended accuracy: ILP {FINAL_OPTIMAL_CONFIG['ilp_accuracy']:.1%}, "
          f"SAT {FINAL_OPTIMAL_CONFIG['sat_accuracy']:.1%}")
    print(f" Unified time limit: {FINAL_OPTIMAL_CONFIG['unified_timeout']}s")
    print(f" Experiment duration: {FINAL_OPTIMAL_CONFIG['experiment_duration_hours']:.2f} hours")
    print("All experiments complete!")
else:
    print("\n Experiment 4.6 failed or was skipped")

print("\n Parameter tweak examples:")
print("EXPERIMENT_4_6_CONFIG['timeout_per_instance'] = 120  # 2 minutes")
print("EXPERIMENT_4_6_CONFIG['max_nodes'] = 20")
print()
print("EXPERIMENT_4_6_CONFIG['timeout_per_instance'] = 1200  # 20 minutes")
print("EXPERIMENT_4_6_CONFIG['max_nodes'] = 50")
print("EXPERIMENT_4_6_CONFIG['skip_large_instances'] = False")


Experiment 4.6: Optimal ILP vs SAT performance comparison (unified config)
Parameters:
  max_nodes: 50
  max_edges: 10000
  min_nodes: 3
  timeout_per_instance: 1200
  buffer_time: 10
  skip_large_instances: True
  enable_progress_display: True
  save_intermediate_results: False
  max_density: 1.0
  min_density: 0.0

 Notes on parameter tweaks:
  To modify parameters, edit the EXPERIMENT_4_6_CONFIG dictionary above
  Key parameters:
    timeout_per_instance: time limit per instance (seconds)
    max_nodes: maximum number of nodes
    max_edges: maximum number of edges
    skip_large_instances: whether to skip large instances

Running Experiment 4.6 with unified configuration (accuracy first)...
Using hardcoded best result from Experiment 4.5:
  Best encoding variant: v5_enhanced_constraints
  Accuracy: 77.1%
  Config name: presolve_only
  Preprocessing: False
  MIPStart: False
  Lazy constraints: False
  Gurobi presolve: 2

Running with unified configuration:
  Time limit: 1200s (20.0m